# connman

A vulnerability in connman was used to hack a [Tesla Model X](https://youtu.be/krSj81thN0w?t=2183). The vulnerability was found using fuzzing. The goal of this exercise is to find such vulnerabilities statically.

We begin by setting up the basics for joern.

In [1]:
import asyncio

from joern_lib import client, workspace
from joern_lib.detectors import common as cpg, c
from joern_lib.utils import print_table

joern_host = "http://joern:9000"
joern_username = "admin"
joern_password  = "admin"

## Clone the vulnerable repo

We clone the vulnerable [repo](https://github.com/helium/connman) using GitPython and importing to joern using import_code api.


In [3]:
import git
import os

if not os.path.exists("/tmp/connman"):
    try:
        git.Repo.clone_from("https://github.com/helium/connman", "/tmp/connman", branch="master", depth=1)
    except Exception as e:
        print(e)
else:
    print("/tmp/connman already exists")

/tmp/connman already exists


In [4]:
async def create_workspace():
    connection = await client.get(joern_host, joern_username, joern_password)
    res = await workspace.import_code(connection, "/tmp/connman", "connman");
    print (res)
asyncio.run(create_workspace())

True


## Practice with C detectors

Joern lib has a detector module with predefined queries for common use cases.

In [6]:
async def test_detectors_fn():
    connection = await client.get(joern_host, joern_username, joern_password)
    res = await cpg.get_complex_functions(connection, n=10)
    print_table (res, title="Complex Functions in code")
    res = await cpg.get_too_many_loops_functions(connection, n=10)
    print_table (res, title="Too Many Loops Functions in code")
    res = await cpg.get_too_nested_functions(connection, n=6)
    print_table (res, title="Too Nested Functions in code")
asyncio.run(test_detectors_fn())

                                                                                                                               Complex Functions in code                                                                                                                                
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ node                                ┃ symbol                              ┃ packageName ┃ nodeLabel ┃ methodShortName                     ┃ methodFullName                      ┃ lineNumber ┃ filename                       ┃ classShortName ┃ className                           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━

Future exception was never retrieved
future: <Future finished exception=ConnectionClosedError(None, None, None)>
Traceback (most recent call last):
  File "/opt/polynote/tmp/joern-notes/connman.ipynb/venv/lib64/python3.9/site-packages/websockets/legacy/protocol.py", line 968, in transfer_data
    message = await self.read_message()
  File "/opt/polynote/tmp/joern-notes/connman.ipynb/venv/lib64/python3.9/site-packages/websockets/legacy/protocol.py", line 1038, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
  File "/opt/polynote/tmp/joern-notes/connman.ipynb/venv/lib64/python3.9/site-packages/websockets/legacy/protocol.py", line 1113, in read_data_frame
    frame = await self.read_frame(max_size)
  File "/opt/polynote/tmp/joern-notes/connman.ipynb/venv/lib64/python3.9/site-packages/websockets/legacy/protocol.py", line 1170, in read_frame
    frame = await Frame.read(
  File "/opt/polynote/tmp/joern-notes/connman.ipynb/venv/lib64/python3.9/site-packages/web

## Detect dangerous string copy functions used

Avoid `strcpy` or `strncpy` function. `strcpy` does not check buffer lengths. A possible mitigation could be `strncpy` which could prevent buffer overflows but does not null-terminate strings leading to memory corruption (Out-of-bounds write). A secure alternative (on BSD) is `strlcpy`.


In [8]:
async def dangerous_string_fn():
    connection = await client.get(joern_host, joern_username, joern_password)
    res = await c.get_strcpy(connection)
    print_table (res, title="Dangerous string functions usage")
asyncio.run(dangerous_string_fn())

                                                                                                                            Dangerous string functions usage                                                                                                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ node                                         ┃ symbol                                       ┃ packageName ┃ nodeLabel ┃ methodShortName                 ┃ methodFullName                  ┃ lineNumber ┃ filename                ┃ classShortName ┃ className                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━

## Detect malloc memcpy overflow

Dangerous copy-operation into heap-allocated buffer.

In [10]:
async def malloc_overflow():
    connection = await client.get(joern_host, joern_username, joern_password)
    res = await client.p(connection, """
        ({val src =
            cpg.method(".*malloc$").callIn.where(_.argument(1).arithmetic).l

            cpg.method("(?i)memcpy").callIn.l.filter { memcpyCall =>
            memcpyCall
                .argument(1)
                .reachableBy(src)
                .where(_.inAssignment.target.codeExact(memcpyCall.argument(1).code))
                .whereNot(_.argument(1).codeExact(memcpyCall.argument(3).code))
                .hasNext
        }}).location.toJsonPretty
    """, title="Malloc memcpy overflow problems")
asyncio.run(malloc_overflow())

                                                                                                                            Malloc memcpy overflow problems                                                                                                                             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ node                                             ┃ symbol                                           ┃ packageName ┃ nodeLabel ┃ methodShortName            ┃ methodFullName             ┃ lineNumber ┃ filename                 ┃ classShortName ┃ className                         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Free follows value reuse

A value that is free'd is reused without reassignment. A value is used after being free'd in a path that leads to it without reassignment. Modeled after CVE-2019-18903.

In [12]:
async def free_reuse():
    connection = await client.get(joern_host, joern_username, joern_password)
    res = await client.p(connection, """
        ({cpg.method
            .name("(.*_)?free")
            .filter(_.parameter.size == 1)
            .callIn
            .where(_.argument(1).isIdentifier)
            .flatMap(f => {
                val freedIdentifierCode = f.argument(1).code
                val postDom             = f.postDominatedBy.toSetImmutable

                val assignedPostDom = postDom.isIdentifier
                .where(_.inAssignment)
                .codeExact(freedIdentifierCode)
                .flatMap(id => id ++ id.postDominatedBy)

                postDom
                .removedAll(assignedPostDom)
                .isIdentifier
                .codeExact(freedIdentifierCode)
            })}).location.toJsonPretty
    """, title="Free follows reuse problems")
asyncio.run(free_reuse())

                                                                                                                              Free follows reuse problems                                                                                                                               
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ node                                                                                                             ┃ symbol    ┃ packageName ┃ nodeLabel  ┃ methodShortName     ┃ methodFullName      ┃ lineNumber ┃ filename           ┃ classShortName ┃ className                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━

## Just show me the CVE!!

Source: Parameter to the method `tcp_server_event`
Sink: Argument from the call `strncpy`

In [14]:
async def find_reachable_flows():
    connection = await client.get(joern_host, joern_username, joern_password)
    await client.flows(connection, 'def source=cpg.method(".*tcp_server_event.*").parameter', 'def sink=cpg.call(".*strncpy.*").argument')
asyncio.run(find_reachable_flows())

________________________________________________________________________________________________________________________________________________________________________________________________________
| nodeType           | tracked                                                                                                                         | lineNumber| method            | file           |
|=======================================================================================================================================================================================================|
| MethodParameterIn  | tcp_server_event(GIOChannel *channel, GIOCondition condition, gpointer user_data)                                               | 2253      | tcp_server_event  | src/dnsproxy.c |
| Identifier         | g_io_channel_unix_get_fd(channel)                                                                                               | 2259      | tcp_server_event  | src/dnsp